# GROUP ASSIGNMENT [COSC2753]

**Members**:
- Vo Thanh Luan – s3822042

# I. Problem statement

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
# Image reprocessing and image extraction
import os
import cv2
import pickle
import tensorflow as tf
import matplotlib.image as mpimg
from tensorflow import keras
from PIL import Image
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

ModuleNotFoundError: No module named 'tensorflow'

# II. Data retrieving

In [ ]:
input_dir = "./data/Flowers/"
flower_categories = os.listdir(input_dir)
print(flower_categories)

In [ ]:
link = []
for label in flower_categories:
    path = os.path.join(input_dir, label) # combine path and labels
    link.append(path) # append in link
print(link)

for i in range(len(link)):
    new = os.listdir(link[i])
    i+=1
    print(f"length : cd {len(new)}") # each folder total image count

Raw images memory is about ~1000kb, which is a very good image for viewing but a very computationally expensive image to fit into CNN models. Therefore, they are downscaled to only 224 x 224 pixels dimension, which is the appropriate data format for Keras CNN models like ResNet and VGG to train.

In [ ]:
data_generators = {}

def process_images():

    for category in flower_categories:
        data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
        ).flow_from_directory(
            input_dir,
            target_size=(224, 224),
            class_mode="categorical",
            shuffle=True,
            classes=[category]
        )
    data_generators[category] = data_generator

In [ ]:
process_images()

In [ ]:
fig_dict = {}
for category in flower_categories:
    fig_dict[category], ax = plt.subplots(ncols=4, figsize=(20,20))
    batch = data_generators[category].next()
    for idx, img in enumerate(batch[0][:4]):
        ax[idx].imshow(img)
        ax[idx].set_title(category)

In [ ]:
total_sizes = {}
for category in flower_categories:
    total_sizes[category] = len(data_generators[category].filenames)

fig, ax = plt.subplots()
ax.bar(total_sizes.keys(), total_sizes.values())

# set chart title and axis labels
ax.set_title("Number Of Images In Each Flower Category")
ax.set_xlabel("Flower Categories")
ax.set_ylabel("Number of Images")

fig.set_size_inches(10, 6)
ax.tick_params(axis='x', labelsize=8)

# show the chart
plt.show()

## Task 2: Recommendation

### Feature engineering

Resnet50 For Feature Extraction

In [ ]:
# display a single image
def display_image(path):
    img = Image.open(path)
    display(img)

Recommender using ResNet50 Features

Recommender using VGG16 Features